In [5]:
import os
import cv2
import numpy as np
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import warnings
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning)

key_training_data = []

class Labeling():        
        
    def make_training_data():
        
        
        keys = ['C','C#','Db','D','D#','Eb','E','F','F#','Gb','G','G#','Ab','A','A#','Bb','B']
        keyIndex = [0, 1, 1, 2, 3, 3, 4, 5, 6, 6, 7, 8, 8, 9, 10, 10, 11]
        keys_counters= [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
        path = "C:/outs/spects4096"
        
        os.chdir(path)
        spectrograms = os.listdir()
        key_count = [0] * 24
        annotations = open("C:/outs/shiftedAnnotations4096.txt", "r")
        #annotations = open("C:/outs/square4096.txt", "r")
        while True: 
            line = annotations.readline()
            if not line:
                break;
            
            
            file, key, chord = line.split(' ', 2)
            file = file.strip()
            key = key.strip()
            chord = chord.strip()
            
            file = file + ".LOFI.png"
            spect_path = os.path.join(path, file)
            spect_path = cv2.imread(spect_path)
           
            if spect_path is None :
                continue;
                
                
            for j,i in enumerate(keys):
                
                temp = keyIndex[j]
                if keys_counters[temp] > 9 : 
                    continue;
                
                if key == i:
                    key_training_data.append([np.array(spect_path)/255, torch.LongTensor([keyIndex[j]])])
                    keys_counters[temp] += 1
           
        
        
        np.random.shuffle(key_training_data)
        #print(keys_counters)
        
if __name__ == '__main__':
    Labeling.make_training_data()
    
   
    

In [9]:
print (key_training_data[1])


[array([[[0.87843137, 0.87843137, 0.87843137],
        [1.        , 1.        , 1.        ],
        [1.        , 1.        , 1.        ],
        ...,
        [1.        , 1.        , 1.        ],
        [1.        , 1.        , 1.        ],
        [0.58823529, 0.58823529, 0.58823529]],

       [[0.87843137, 0.87843137, 0.87843137],
        [1.        , 1.        , 1.        ],
        [1.        , 1.        , 1.        ],
        ...,
        [1.        , 1.        , 1.        ],
        [1.        , 1.        , 1.        ],
        [0.58823529, 0.58823529, 0.58823529]],

       [[0.87843137, 0.87843137, 0.87843137],
        [1.        , 1.        , 1.        ],
        [1.        , 1.        , 1.        ],
        ...,
        [1.        , 1.        , 1.        ],
        [1.        , 1.        , 1.        ],
        [0.58431373, 0.58431373, 0.58431373]],

       ...,

       [[0.16078431, 0.16078431, 0.16078431],
        [0.07843137, 0.07843137, 0.07843137],
        [0.10196078, 

In [ ]:
#KEY ESTIMATION NEURAL NETWORK

import torch
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 8, 5)
        self.conv2 = nn.Conv2d(8, 8, 5)
        self.conv3 = nn.Conv2d(8, 8, 5)
        self.conv4 = nn.Conv2d(8, 8, 5)
        self.conv5 = nn.Conv2d(8, 8, 5)

        
        x = torch.randn(3,105,600).view(-1,3,105,600)
        self._to_linear = None
        self.convs(x)
        self.fc1 = nn.Linear(self._to_linear, 1024)
        self.fc2 = nn.Linear(1024, 64)
        self.fc3 = nn.Linear(64, 12)
        self.dropout = nn.Dropout(0.5)
        
    def convs(self, x):
        x = F.elu(self.conv1(x))
        x = F.elu(self.conv2(x))
        x = F.elu(self.conv3(x))
        x = F.elu(self.conv4(x))
        x = F.avg_pool2d(F.elu(self.conv5(x)), (2,2))
        
                
        if self._to_linear is None:
            self._to_linear = x[0].shape[0]*x[0].shape[1]*x[0].shape[2]
        return x
        
    
    def forward(self, x):
        x = self.convs(x)
        x = x.view(-1, self._to_linear)
        x = F.elu(self.fc1(x))
        x = self.dropout(x)
        x = F.elu(self.fc2(x))
        x = self.fc3(x)
        return x
    
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    k_Net = Net().to(device)


print(k_Net)


In [ ]:
#SETTING OPTIMIZER
import torch.optim as optim

lr = 0.001
optimizer = torch.optim.SGD(k_Net.parameters(), lr= lr , momentum = 0.9, weight_decay = 0.0001) 
loss_function = nn.CrossEntropyLoss()
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    loss_function = loss_function.to(device)

print("optimizer komple ")

In [ ]:
#Helpfull functions for training 


def lr_handler(loss, change_again):
    """an to loss exei pesei katw apo mia timh tote allazoume to lr"""
    
    if (loss < 0.7) & (change_again) :
         
        lr = 0.0005
        optimizer = torch.optim.SGD(k_Net.parameters(), lr = lr, momentum = 0.9, weight_decay = 0.0001) 
        print(f"\nlr changed cause of very low loss: {loss} \nNew lr: {lr} \n" )
        print("------------------------------------------------------------")
        change_again = False
        
    return change_again
        
def train_evaluation (loader, epochs):
    """trexoume ena mini test ana 5 epochs"""
    acc = 101
    correct, total = 0, 0

    if (((epochs+1) % 5) == 0) :

        k_Net.eval()

        with torch.no_grad():
            for i, test_data in enumerate(loader, 0):
                # prepare testing data 
                test_inputs = test_data[0].view(-1,3,105,600)
                test_targets = test_data[1]
                test_inputs = test_inputs.float()
                
                # Set data to GPU
                test_inputs, test_targets = test_inputs.to(device), test_targets.to(device)
                # Generate outputs
                test_outputs = k_Net(test_inputs)

                # Set total and correct
                _, predicted = torch.max(test_outputs.data, 1)
                total += test_targets.size(0)
                
                cntr = 0
                for i in predicted:
                    if i == test_targets[cntr]:
                        correct += 1
                    cntr += 1

                    
        acc = (correct / total)*100

    k_Net.train()
    return acc 

In [ ]:
#GET DATA DROM DISK key_1500_training_data.npy 
import numpy as np

key_training_data = np.load("C:/outs/spects4096/all_key_training_data.npy", allow_pickle= True)

In [ ]:
# mia fora 8a trexei auto. Metratoph se tensor kai apo8hkeush tou se tensor file 

#for i, data in enumerate(key_training_data): #ΑYTO EDW 8A STA KANEI TENSOR TA DATA SOU ALLA KANTA KAI ENA SAVE TO TENSOR https://discuss.pytorch.org/t/save-a-tensor-to-file/37136
    #key_training_data[0] = torch.from_numpy(data[0])
    #print(type(data[0]))    

In [ ]:
print(type(key_training_data[0][0][0]))

In [ ]:
# PREPARE DATA 
from torch.utils.data import DataLoader

val_size = int(len(key_training_data)*0.1)
training_data = key_training_data[:len(key_training_data)-val_size]
test_data = key_training_data[-val_size:]

batchSize = 16
train_dataloader = DataLoader(training_data, batch_size=batchSize , shuffle=True, pin_memory=True, drop_last=True)
test_dataloader = DataLoader(test_data, batch_size = batchSize, shuffle=True, drop_last=True)


In [ ]:
#TRAINING KEY NEURAL NETWORK


k_Net.train()
num_epochs = 20
acc = 101 
our_acc = 0
flag = True # flag for not changing again and again the lr when loss remains below 0.5 

for epoch in range(0, num_epochs):
    print("-----------------------------------------------------------")
    
    for i, data in enumerate(train_dataloader, 0):
        # Get inputs
        inputs = data[0].view(-1,3,105,600)
        targets = data[1]
        inputs = inputs.float()

        inputs, targets = inputs.to(device), targets.to(device)

        optimizer.zero_grad()
        
        # Generate outputs
        outputs = k_Net(inputs)
        
        # Set total and correct
        targets = targets.view(len(data[1]))
        
        loss = loss_function(outputs, targets)
        loss.backward()
        optimizer.step()
        
    
    flag = lr_handler(loss, flag) # check learning rate
    acc = train_evaluation(test_dataloader, epoch) # evaluate the model every epoch 
    
    if acc != 101:
        
        print("| Epoch : %2d | Loss: %.5f | currenct accuracy: %2.5f |" % (epoch, loss, acc))
        if (our_acc < acc): # if the accuracy imporves save the models parameteres
            our_acc = acc
            torch.save(k_Net.state_dict(), 'C:/Users/Michalis Zeakis/Desktop/university/ptyxiaki/saved_nns/test1-best-key-network-model-parameters1.pt')
        acc = 101
    else:
        print("| Epoch : %2d | Loss: %.5f | " % (epoch ,loss))

In [ ]:
torch.save(k_Net.state_dict(), 'C:/Users/Michalis Zeakis/Desktop/university/ptyxiaki/saved_nns/best-key-network-model-parameters-overfitted.pt')

In [ ]:
#TESTING KEY NEURAL NETWORK

correct, total = 0, 0
batchSize = 6
test_data = key_training_data[-val_size:]
test_dataloader = DataLoader(test_data, batch_size = batchSize, shuffle=True, drop_last=True)

k_Net.eval()

print("``` Starting Testing ```")

with torch.no_grad():
    print("kaispera")
    

    # Iterate over the test data and generate predictions
    for i, data in enumerate(test_dataloader, 0):

    # Get inputs
        inputs = data[0].view(-1,3,105,600)
        targets = data[1]
        inputs = inputs.float()

        inputs, targets = inputs.to(device), targets.to(device)
    # Generate outputs
        outputs = k_Net(inputs)

    # Set total and correct

        _, predicted = torch.max(outputs.data, 1)
        
        total += targets.size(0)
        
        cntr = 0
        for i in predicted:# counter for correct predictions

            if i == targets[cntr]:
                correct += 1
            cntr += 1

        
    
print("Accuracy: ", 100*(correct/total))



In [ ]:
print(correct)
print(total)
print(our_acc)


In [ ]:
#TESTING ON TRAINING DATA GIA NA TSEKAROUME OVERFITTING


correct, total = 0, 0
batchSize = 32
overfit = DataLoader(training_data , batch_size = batchSize, shuffle=True, drop_last=True)

k_Net.eval()

print("``` Starting Testing overfit ```")
with torch.no_grad():
    #running_loss = 0.0
    #loss_values = [] 

    # Iterate over the test data and generate predictions
    for i, data in enumerate(overfit, 0):
    # Get inputs
        inputs = data[0].view(-1,3,105,600)
        targets = data[1]
        inputs = inputs.float()

        inputs, targets = inputs.to(device), targets.to(device)
    # Generate outputs
        outputs = k_Net(inputs)

    # Set total and correct

        _, predicted = torch.max(outputs.data, 1)
        total += targets.size(0)
        #running_loss =+ loss.item() * batchSize
        #correct += (predicted == targets).sum().item()
        cntr = 0
        for i in predicted:

            if i == targets[cntr]:
                correct += 1
            cntr += 1

        #loss_values.append(running_loss / len(test_data))
    
print("Accuracy: ", 100*(correct/total))

#plt.plot(loss_values)




In [ ]:
print(correct)
print(total)

In [ ]:
# EVALUATION AT SMALL DATA TRAINED MODEL 


model = Net()
model = model.to(device)
model.load_state_dict(torch.load('C:/Users/Michalis Zeakis/Desktop/university/ptyxiaki/saved_nns/288-best-key-network-model-parameters.pt'))


correct, total = 0, 0
batchSize = 32
overfit = DataLoader(key_training_data , batch_size = batchSize, shuffle=True, drop_last=True)

k_Net.eval()

print("``` Starting Testing ```")
with torch.no_grad():
    #running_loss = 0.0
    #loss_values = [] 

    # Iterate over the test data and generate predictions
    for i, data in enumerate(overfit, 0):
    # Get inputs
        inputs = data[0].view(-1,3,105,600)
        targets = data[1]
        inputs = inputs.float()

        inputs, targets = inputs.to(device), targets.to(device)
    # Generate outputs
        outputs = model(inputs)

    # Set total and correct

        _, predicted = torch.max(outputs.data, 1)
        total += targets.size(0)
        #running_loss =+ loss.item() * batchSize
        #correct += (predicted == targets).sum().item()
        cntr = 0
        for i in predicted:

            if i == targets[cntr]:
                correct += 1
            cntr += 1

        #loss_values.append(running_loss / len(test_data))
    
print("Accuracy: ", 100*(correct/total))


In [ ]:
print(correct)
print(total)

In [ ]:
K_model = k_Net
K_model.load_state_dict(torch.load("C:/Users/Michalis Zeakis/Desktop/university/ptyxiaki/saved_nns/best-key-network-model-parameters-overfitted.pt"))

In [ ]:
from torch.utils.data import DataLoader

#TESTING ON TRAINING DATA GIA NA TSEKAROUME OVERFITTING


correct, total = 0, 0
batchSize = 32
overfit = DataLoader(key_training_data , batch_size = batchSize, shuffle=True, drop_last=True)

k_Net.eval()

print("``` Starting Testing overfit ```")
with torch.no_grad():
    #running_loss = 0.0
    #loss_values = [] 

    # Iterate over the test data and generate predictions
    for i, data in enumerate(overfit, 0):
    # Get inputs
        inputs = data[0].view(-1,3,105,600)
        targets = data[1]
        inputs = inputs.float()

        inputs, targets = inputs.to(device), targets.to(device)
    # Generate outputs
        outputs = k_Net(inputs)

    # Set total and correct

        _, predicted = torch.max(outputs.data, 1)
        total += targets.size(0)
        #running_loss =+ loss.item() * batchSize
        #correct += (predicted == targets).sum().item()
        cntr = 0
        for i in predicted:

            if i == targets[cntr]:
                correct += 1
            cntr += 1

        #loss_values.append(running_loss / len(test_data))
    
print("Accuracy: ", 100*(correct/total))

#plt.plot(loss_values)


